In [1]:
#!/usr/bin/env python

import os
import pandas as pd
import numpy as np
from datetime import datetime

# ============================================================
# CONFIG
# ============================================================

EQUITY_FILE = "./spy_backtest_output/system1_breakout_equity.csv"
TRADES_FILE = "./spy_backtest_output/system1_breakout_trades.csv"

OUTPUT_DIR = "./spy_backtest_cost_tests"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SLIPPAGE_BPS_LIST = [1, 3, 5, 10]   # basis points
COMMISSION = 9.99                  # per BUY or SELL
TRADING_DAYS_PER_YEAR = 252


# ============================================================
# PERFORMANCE HELPERS
# ============================================================

def cagr(eq):
    start_val = eq.iloc[0]
    end_val   = eq.iloc[-1]
    years = len(eq) / TRADING_DAYS_PER_YEAR
    return (end_val / start_val) ** (1/years) - 1

def sharpe(returns):
    return returns.mean() / returns.std() * np.sqrt(TRADING_DAYS_PER_YEAR)

def sortino(returns):
    downside = returns[returns < 0]
    if downside.std() == 0:
        return np.nan
    return returns.mean() / downside.std() * np.sqrt(TRADING_DAYS_PER_YEAR)

def max_drawdown(eq):
    roll_max = eq.cummax()
    dd = eq / roll_max - 1
    return dd.min()


# ============================================================
# LOAD DATA
# ============================================================

print("\n=== LOADING SPY BREAKOUT DATA ===")

eq_base = pd.read_csv(EQUITY_FILE)
eq_base["date"] = pd.to_datetime(eq_base["date"])
eq_base = eq_base.sort_values("date")

trades = pd.read_csv(TRADES_FILE)
trades["date"] = pd.to_datetime(trades["date"])
trades = trades.sort_values("date")

print(f"Loaded equity: {len(eq_base):,} rows")
print(f"Loaded trades: {len(trades):,} trades")


# ============================================================
# BASELINE METRICS
# ============================================================

eq_series_base = eq_base["equity"]

base_cagr = cagr(eq_series_base)
base_sharpe = sharpe(eq_series_base.pct_change().dropna())
base_sortino = sortino(eq_series_base.pct_change().dropna())
base_mdd = max_drawdown(eq_series_base)

print("\n=== BASELINE (NO COSTS) ===")
print(f"CAGR:        {base_cagr:.4f}")
print(f"Sharpe:      {base_sharpe:.4f}")
print(f"Sortino:     {base_sortino:.4f}")
print(f"MaxDD:       {base_mdd:.4f}")


# ============================================================
# APPLY TRANSACTION COSTS
# ============================================================

results = []

for bps in SLIPPAGE_BPS_LIST:

    print(f"\n--- Testing slippage = {bps} bp + ${COMMISSION} commission per trade ---")

    # Start with baseline equity curve
    eq = eq_base.copy()
    eq["equity_after_costs"] = eq["equity"].copy()

    # Total slippage cost accumulator
    total_commission_cost = 0
    total_slippage_cost = 0

    # Apply costs trade-by-trade
    for i, t in trades.iterrows():

        trade_date = t["date"]
        price = t["price"]
        shares = t["shares"]

        # Commission
        cost_comm = COMMISSION
        total_commission_cost += cost_comm

        # Slippage
        slip_pct = bps / 10000.0   # convert bp → percent
        cost_slippage = price * slip_pct * shares
        total_slippage_cost += cost_slippage

        # Reduce equity starting from this trade's date forward
        eq.loc[eq["date"] >= trade_date, "equity_after_costs"] -= (cost_comm + cost_slippage)

    # Compute metrics
    eq_cost = eq["equity_after_costs"]
    ret_cost = eq_cost.pct_change().dropna()

    cagr_cost = cagr(eq_cost)
    sharpe_cost = sharpe(ret_cost)
    sortino_cost = sortino(ret_cost)
    mdd_cost = max_drawdown(eq_cost)

    print(f"Trades:              {len(trades):,}")
    print(f"Total commission:    ${total_commission_cost:,.2f}")
    print(f"Total slippage:      ${total_slippage_cost:,.2f}")
    print(f"CAGR (after costs):  {cagr_cost:.4f}")
    print(f"Sharpe (after costs):{sharpe_cost:.4f}")
    print(f"Sortino (after):     {sortino_cost:.4f}")
    print(f"MaxDD  (after):      {mdd_cost:.4f}")

    # Save equity curve
    out_parquet = os.path.join(
        OUTPUT_DIR,
        f"equity_after_costs_{bps}bp_{datetime.now().strftime('%Y%m%d-%H%M%S')}.parquet"
    )
    eq.to_parquet(out_parquet, index=False)
    print(f"Saved equity curve with costs ({bps} bp) → {out_parquet}")

    # Store results
    results.append({
        "slippage_bp": bps,
        "total_trades": len(trades),
        "total_commission": total_commission_cost,
        "total_slippage": total_slippage_cost,
        "cagr_after_costs": cagr_cost,
        "sharpe_after_costs": sharpe_cost,
        "sortino_after_costs": sortino_cost,
        "maxdd_after_costs": mdd_cost,
        "baseline_cagr": base_cagr,
        "baseline_sharpe": base_sharpe,
        "baseline_sortino": base_sortino,
        "baseline_maxdd": base_mdd
    })


# ============================================================
# SAVE SUMMARY TABLE
# ============================================================

summary = pd.DataFrame(results)
summary_path = os.path.join(
    OUTPUT_DIR,
    f"transaction_cost_summary_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv"
)
summary.to_csv(summary_path, index=False)

print("\n=== SUMMARY ===")
print(summary)

print(f"\nSummary saved → {summary_path}")
print("\n=== TRANSACTION-COST TEST COMPLETE ===\n")



=== LOADING SPY BREAKOUT DATA ===
Loaded equity: 7,023 rows
Loaded trades: 79 trades

=== BASELINE (NO COSTS) ===
CAGR:        0.0721
Sharpe:      0.6912
Sortino:     0.7324
MaxDD:       -0.2906

--- Testing slippage = 1 bp + $9.99 commission per trade ---
Trades:              79
Total commission:    $789.21
Total slippage:      $8,512.70
CAGR (after costs):  0.0720
Sharpe (after costs):0.6886
Sortino (after):     0.7300
MaxDD  (after):      -0.2923
Saved equity curve with costs (1 bp) → ./spy_backtest_cost_tests\equity_after_costs_1bp_20251202-184115.parquet

--- Testing slippage = 3 bp + $9.99 commission per trade ---
Trades:              79
Total commission:    $789.21
Total slippage:      $25,538.11
CAGR (after costs):  0.0718
Sharpe (after costs):0.6840
Sortino (after):     0.7252
MaxDD  (after):      -0.2950
Saved equity curve with costs (3 bp) → ./spy_backtest_cost_tests\equity_after_costs_3bp_20251202-184115.parquet

--- Testing slippage = 5 bp + $9.99 commission per trade ---